**Everyone go upvote [David Austin's](https://www.kaggle.com/code/tivfrvqhs5/decode-jpeg2000-dicom-with-dali) notebook that showed us the amazing powers of DALI! This a fork of his notebook. For modeling, I set up my local kernel based on [Theo Viel's](https://www.kaggle.com/code/theoviel/rsna-breast-baseline-faster-inference-with-dali) inference kernel, please go up vote this notebook too. They're both amazing and I learned a lot!**

There's more improvements, I am seeing if we can get rid of the last I/O action after cropping but I did not end up doing it here because it will greatly restrict reuseability.

About 3.8h runtime (compared to Theo Viel's non full dali pipeline, but this one includes YoloV5 ROI)

In [1]:
!pip install -I --no-index --find-links /kaggle/input/offline-rsna-dali-pipeline-pips python-gdcm pydicom pylibjpeg dicomsdl nvidia-dali-nightly-cuda110

Looking in links: /kaggle/input/offline-rsna-dali-pipeline-pips
Processing /kaggle/input/offline-rsna-dali-pipeline-pips/python_gdcm-3.0.20-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/offline-rsna-dali-pipeline-pips/pydicom-2.3.1-py3-none-any.whl
Processing /kaggle/input/offline-rsna-dali-pipeline-pips/pylibjpeg-1.4.0-py3-none-any.whl
Processing /kaggle/input/offline-rsna-dali-pipeline-pips/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
Processing /kaggle/input/offline-rsna-dali-pipeline-pips/nvidia_dali_nightly_cuda110-1.22.0.dev20230102-6912348-py3-none-manylinux2014_x86_64.whl
Processing /kaggle/input/offline-rsna-dali-pipeline-pips/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
Processing /kaggle/input/offline-rsna-dali-pipeline-pips/gast-0.4.0-py3-none-any.whl
Processing /kaggle/input/offline-rsna-dali-pipeline-pips/astunparse-1.6.3-py2.py3-none-any.whl
Processing /kaggle/input/offline-rsna-

In [2]:
import os
import gc
import sys
import glob

import cupy as cp
import numpy as np
import pandas as pd

import cv2
import torch
import pydicom
import dicomsdl
import matplotlib

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from builtins import StopIteration
from joblib import Parallel, delayed
from pydicom.filebase import DicomBytesIO

import torch.nn as nn
import albumentations as albu
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader

import nvidia.dali.fn as fn
import nvidia.dali.ops as ops
import nvidia.dali.types as types
from nvidia.dali import pipeline_def
from nvidia.dali.pipeline import Pipeline
from nvidia.dali.types import DALIDataType
from nvidia.dali.plugin.pytorch import DALIGenericIterator
from nvidia.dali.plugin.numba.fn.experimental import numba_function

sys.path.append('/kaggle/input/yolov5-github-repo-tracker/yolov5')
sys.path.append('/kaggle/input/timm-0-6-9/pytorch-image-models-master')

import timm

from utils.general import non_max_suppression

In [3]:
class Config:
    # dirs
    img_dir = '../input/rsna-breast-cancer-detection/test_images'
    oimg_dir = '/tmp/imgs' # output image dir
    
    # df
    df_path = '/kaggle/input/rsna-breast-cancer-detection/test.csv'
    
    # roi pipeline 
    batch_size = 32
    resize_height_to = 640 # yolov5 image size
    factor = 2 # image downscale factor - saves memory
    out_img_size = 1024 # output image size -> outputs ratio 1024x512
    
    # dali pipeline args
    pipeline_args = {
        'batch_size': 32,
        'num_threads': 2,
        'device_id': 0,
        'py_num_workers': 4,
        'exec_async': False,
        'exec_pipelined': False
    }
    
    # yolov5
    yolov5_model = '/kaggle/input/rsna-yolov5-padded/best.pt'
    yolov5_path = '/kaggle/input/yolov5-github-repo-tracker/yolov5'
    
    # dataloader
    num_workers = 2
    cnn_batch_size = 16
    model_name = 'tf_efficientnet_b0_ns'
    
    threshold = [0.28, 0.23]
    
    debug = False

In [4]:
def __read_metadata(fn):
    f = open(fn, 'rb')
    md = f.read(400)
    f.close()
    offset = md.index(b'1.2.840.10008.1.2.4.') + 20
    return md[offset] == 57

In [5]:
df = pd.read_csv(Config.df_path)

if Config.debug:
    df = df.sample(1000).reset_index(drop = True)

df['pis'] = Config.img_dir + '/' + df.patient_id.astype(str) + '/' + df.image_id.astype(str) + '.dcm'
df['path']= Config.oimg_dir + '/' + df.patient_id.astype(str) + '_' + df.image_id.astype(str) + '.png'
# low overhead so we can increase n_jobs to max cpu usage
is_j2k = np.array(Parallel(n_jobs = 24)(delayed(__read_metadata)(ipath) for ipath in tqdm(df.pis.values.tolist())))
df_j2k = df.loc[is_j2k]
df_jll = df.loc[~is_j2k]

os.makedirs(Config.oimg_dir)

print('j2k:', len(df_j2k), 'jll:', len(df_jll))

model = torch.hub.load(
    Config.yolov5_path, 'custom', path = Config.yolov5_model, source = 'local', force_reload = True
).to('cuda')

  0%|          | 0/4 [00:00<?, ?it/s]

j2k: 4 jll: 0


fatal: unsafe repository ('/kaggle/input/yolov5-github-repo-tracker/yolov5' is owned by someone else)
To add an exception for this directory, call:

	git config --global --add safe.directory /kaggle/input/yolov5-github-repo-tracker/yolov5
YOLOv5 🚀 2023-1-3 Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model summary: 157 layers, 1765028 parameters, 0 gradients
Adding AutoShape... 


In [6]:
class J2KIterator(object):
    def __init__(self, df, batch_size):
        self.df = df
        self.batch_size = batch_size
       
    @staticmethod
    def convert_dicom_to_j2k(patient_id, image_id):
        dcmfile = pydicom.dcmread(f'{Config.img_dir}/{patient_id}/{image_id}.dcm')
        with open(f'{Config.img_dir}/{patient_id}/{image_id}.dcm', 'rb') as fp:
            raw = DicomBytesIO(fp.read())
            ds = pydicom.dcmread(raw)
        offset = ds.PixelData.find(b"\x00\x00\x00\x0C")

        return np.frombuffer(ds.PixelData[offset:], dtype=np.uint8), dcmfile.PhotometricInterpretation == 'MONOCHROME1'
        
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __iter__(self):
        self.i = 0
        return self

    def __next__(self):
        if self.i > len(self.df):
            raise StopIteration
            
        compressed_imgs = []
        is_monochrome_imgs = []
        p_ids = []
        i_ids = []
        
        df = self.df.iloc[range(self.i, min(self.i + self.batch_size, len(self.df)))]
        
        for p_id, i_id in df[['patient_id', 'image_id']].values:
            compressed_img, is_monochrome_img = self.convert_dicom_to_j2k(p_id, i_id)
            
            compressed_imgs.append(compressed_img)
            is_monochrome_imgs.append(np.array([is_monochrome_img], dtype = np.bool_))
            p_ids.append(np.array([p_id], dtype = np.int64)) 
            i_ids.append(np.array([i_id], dtype = np.int64))
            
        self.i += self.batch_size
            
        return compressed_imgs, is_monochrome_imgs, p_ids, i_ids
    
class JLLIterator(object):
    def __init__(self, df, batch_size):
        self.df = df
        self.batch_size = batch_size
        
    @staticmethod
    def norm_img(img):
        i_max = img.max()
        i_min = img.min()
        return (255 * ((m - i_min) / (i_max - i_min))).astype(np.uint8)
       
    @staticmethod
    def decompress_jll(dm):
        info = dm.getPixelDataInfo()
        img = np.empty((info['Rows'], info['Cols']), dtype = info['dtype'])
        dm.copyFrameData(0, img)
        return img
    
    def process_img(self, patient_id, image_id):
        dm = dicomsdl.open(f'{Config.img_dir}/{patient_id}/{image_id}.dcm')
        
#         img = norm_img(decompress_jll(dm))
        
#         if dm.PhotometricInterpretation== 'MONOCHROME1':
#             img = 255 - img
        
#         return img

        return self.decompress_jll(dm).astype(np.uint16), dm.PhotometricInterpretation == 'MONOCHROME1'
        
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __iter__(self):
        self.i = 0
        return self

    def __next__(self):
        if self.i > len(self.df):
            raise StopIteration
        
        decompressed_imgs = []
        is_monochrome_imgs = []
        p_ids = []
        i_ids = []
        
        df = self.df.iloc[range(self.i, min(self.i + self.batch_size, len(self.df)))]
        
        for p_id, i_id in df[['patient_id', 'image_id']].values:
            decompressed_img, is_monochrome_img = self.process_img(p_id, i_id)
            
            decompressed_imgs.append(np.expand_dims(decompressed_img, axis = 2))
            is_monochrome_imgs.append(np.array([is_monochrome_img], dtype = np.bool_))
            p_ids.append(np.array([p_id], dtype = np.int64))
            i_ids.append(np.array([i_id], dtype = np.int64))
            
        self.i += self.batch_size
                    
        return decompressed_imgs, is_monochrome_imgs, p_ids, i_ids

In [7]:
J2Ki = J2KIterator(df_j2k, Config.batch_size)
JLLi = JLLIterator(df_jll, Config.batch_size)

In [8]:
def normalize_img(in_img):
    return (255 * (in_img - in_img.min()) / (in_img.max() - in_img.min())).astype(cp.uint8)

def mux(condition, true_case, false_case):
    neg_condition = condition ^ True
    return condition * true_case + neg_condition * false_case

def common_pipeline(imgs, is_monochromes, p_ids, i_ids):
    # to lower information loss
    imgs = fn.resize(imgs, resize_y = int(Config.resize_height_to * Config.factor), interp_type = types.INTERP_NN)
    
    # normalize
    imgs = fn.cast(imgs, dtype = types.INT32) # weird bug with python_function and unsigned ints
    imgs = fn.python_function(imgs, device = 'gpu', function = normalize_img, batch_processing = False)

    # monochrome invert using mux
    imgs_inverted = 255 - imgs
    imgs = mux(is_monochromes, imgs_inverted, imgs)
    imgs = fn.cast(imgs, dtype = types.UINT8)
    
    # pad to the right to be square
    imgs = fn.pad(imgs, axes = (1,), shape = (int(Config.resize_height_to * Config.factor)), )
    
    # resize for model
    imgs_2roi = fn.resize(imgs, resize_y = Config.resize_height_to) / 255
    
    return imgs_2roi, imgs, p_ids, i_ids

In [9]:
@pipeline_def
def j2k_decode_pipeline():
    imgs, is_monochromes, p_ids, i_ids = fn.external_source(
        source = J2Ki, num_outputs = 4, device = "cpu", dtype = [types.UINT8, types.BOOL, types.INT64, types.INT64],
    )
    
    imgs = fn.experimental.decoders.image(
        imgs, device = 'mixed', output_type = types.ANY_DATA, dtype = DALIDataType.UINT16
    )
    
    return common_pipeline(imgs, is_monochromes, p_ids, i_ids)

@pipeline_def
def jll_process_pipeline():
    imgs, is_monochromes, p_ids, i_ids = fn.external_source(
        source = JLLi, num_outputs = 4, device = "gpu", dtype = [types.UINT16, types.BOOL, types.INT64, types.INT64],
    )
    
    imgs = fn.reinterpret(imgs, layout = "HWC")
    
    return common_pipeline(imgs, is_monochromes, p_ids, i_ids)

In [10]:
pipes = [
    (j2k_decode_pipeline, len(J2Ki)),
    (jll_process_pipeline, len(JLLi))
]

In [11]:
def _p(d):
    if len(d) > 0 and len(d[0]) >= 1:
        return d[0].numpy()
    return np.array([0, 0, Config.resize_height_to, Config.resize_height_to, 1, 0])

for pipe_l, p_len in pipes:
    print("Starting pipeline:", pipe_l.__name__)
    
    if p_len == 0:
        continue
    
    pipe = pipe_l(**Config.pipeline_args)
    pipe.start_py_workers()
    dali_iter = DALIGenericIterator(pipe, ['imgs_2roi', 'imgs', 'p_ids', 'i_ids'])
    
    for i, data in tqdm(enumerate(dali_iter), total = p_len):
        imgs_2roi = data[0]['imgs_2roi']
        imgs = data[0]['imgs'].cpu().numpy()
        p_ids = data[0]['p_ids'].cpu().numpy()
        i_ids = data[0]['i_ids'].cpu().numpy()

        with torch.no_grad():
            y = model(imgs_2roi.expand(-1, -1, -1, 3).permute(0, 3, 1, 2)).cpu()
            dets = non_max_suppression(y, conf_thres = 0.1, classes = [0, 1])
            # dets -> 32 x (x y x y conf cls)

            # keep best 
            dets = np.array([
                _p(d) for d in dets
            ])

        cls_ = dets[:, 4].astype(np.bool_)
        xyxy = dets[:, [0, 1, 2, 3]] / 640
        xyxy = np.clip(xyxy, 0, 1)

        for img, (xmin, ymin, xmax, ymax), lr, p_id, i_id in zip(imgs, xyxy, cls_, p_ids, i_ids):
            h, w, _ = img.shape
            img = np.squeeze(img)

            xmin = (xmin * w).astype(np.uint16)
            xmax = (xmax * w).astype(np.uint16)
            ymin = (ymin * h).astype(np.uint16)
            ymax = (ymax * h).astype(np.uint16)

            if lr:
                img = np.fliplr(img)
                xmin = w - xmin
                xmax = w - xmax
                xmin, xmax = xmax, xmin

            resulting_crop_h = ymax - ymin
            resulting_crop_w = xmax - xmin

            if (2 * resulting_crop_w) < resulting_crop_h: # increase width
                # how much more width do I need to add?
                needed_width = (resulting_crop_h // 2) - resulting_crop_w

                # can I meet width without going past image?
                if needed_width + xmax > w:
                    # padding/resizing is required (I use padding, in this case)
                    difference = xmax + needed_width - w

                    img = cv2.copyMakeBorder(img, 0, 0, difference, 0, cv2.BORDER_CONSTANT, value = 0)

                    # no need to offset bbox since origin is top left

                # expand bbox by needed width
                xmax += needed_width

            elif (2 * resulting_crop_w) > resulting_crop_h: # increase height
                # how much more height do I need to add?
                needed_height = (resulting_crop_w * 2) - resulting_crop_h

                # can I meet height without going past image?
                if ymin - needed_height < 0:
                    # padding/resizing is required (I use padding in this case)
                    difference = needed_height - ymin

                    img = cv2.copyMakeBorder(img, difference, 0, 0, 0, cv2.BORDER_CONSTANT, value = 0)

                    # offset bbox values to new origin
                    ymin += difference
                    ymax += difference

                ymin -= needed_height

            if lr:
                img = np.fliplr(img)
                h, w = img.shape

                xmin = w - xmin
                xmax = w - xmax
                xmin, xmax = xmax, xmin

            img = img[ymin:ymax, xmin:xmax]

            h, w = img.shape

            img = cv2.resize(
                img, dsize = (Config.out_img_size // 2, Config.out_img_size), interpolation=cv2.INTER_NEAREST
            )
            
            cv2.imwrite(f'{Config.oimg_dir}/{p_id[0]}_{i_id[0]}.png', img) # {img_dir}
            
    del img, imgs, xyxy, cls_, p_ids, i_ids, y, dets, data, dali_iter, pipe
    gc.collect()
    
del pipes, model
gc.collect()

Starting pipeline: j2k_decode_pipeline


  0%|          | 0/1 [00:00<?, ?it/s]

Starting pipeline: jll_process_pipeline


2101

In [12]:
transforms = albu.Compose(
    [
        albu.Normalize(mean=0, std=1),
        ToTensorV2(),
    ],
    p=1,
)

class BreastDataset(Dataset):
    def __init__(self,df, transforms=transforms):
        self.paths = df['path'].values
        self.transforms = transforms

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        image = cv2.imread(self.paths[idx], cv2.IMREAD_GRAYSCALE) # 1 channel
        
        if self.transforms:
            image = self.transforms(image=image)["image"]
        
        return image
    
class SimpleCNN(nn.Module):
    def __init__(self, encoder):
        super().__init__()
        
        self.encoder = encoder
        
        self.output_features = encoder.num_features
        
        self.logits = nn.Linear(self.output_features, 1) # cancer or no cancer
        
        self._to_grayscale()
        
    def _to_grayscale(self):
        for n, m in self.encoder.named_modules():
            if n:
                old_conv = getattr(self.encoder, n)
                old_conv_weight = old_conv.weight.data.sum(1, keepdim=True)
                new_conv = nn.Conv2d(
                    1,
                    old_conv.out_channels,
                    kernel_size=old_conv.kernel_size,
                    stride=old_conv.stride,
                    padding=old_conv.padding,
                    bias=old_conv.bias is not None,
                )
                new_conv.weight.data = old_conv_weight.repeat(1, 1, 1, 1)
                setattr(self.encoder, n, new_conv)
                return
            
    def extract_features(self, x):
        fts = self.encoder.forward_features(x)
        
        while len(fts.size()) > 2:
            fts = fts.mean(-1)
            
        return fts

    def get_logits(self, features):
        return self.logits(features)
    
    def forward(self, x):
        x = self.extract_features(x)
        x = self.get_logits(x)
        
        return x
    
def get_model(name):
    backbone = getattr(timm.models, name)(pretrained = False)
    backbone.name = name
    
    model = SimpleCNN(backbone)
    
    return model

def predict(model, dataloader):
    model.eval()

    preds = []
    
    for img in tqdm(dataloader):
        img = img.cuda()

        with torch.no_grad():
            pred = model(img)

        preds.append(pred.sigmoid().cpu().numpy())

    return np.nan_to_num(np.concatenate(preds))

In [13]:
preds = []

model = get_model(Config.model_name).cuda()
dataset = BreastDataset(df)
dataloader = DataLoader(dataset, batch_size = Config.cnn_batch_size, shuffle = False, num_workers = Config.num_workers)

for model_weights in glob.glob('/kaggle/input/rsna-model-weights/*.pth'):
    state_dict = torch.load(model_weights, map_location="cpu")
    model.load_state_dict(state_dict)
    
    pred = predict(model, dataloader)
    preds.append(pred)

preds = np.mean(preds, 0)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
df['cancer'] = preds
df['prediction_id'] = df['patient_id'].astype(str) + '_' + df['laterality']

sub = df[['prediction_id', 'cancer', 'site_id']].groupby("prediction_id").mean().reset_index()
sub['site_id'] = sub['site_id'].astype(int)

if isinstance(Config.threshold, list):
    sub["cancer"] = sub.apply(lambda r: r.cancer > Config.threshold[r.site_id - 1], axis=1).astype(int)
else:
    sub["cancer"] = (sub["cancer"] > Config.threshold).astype(int)

sub[['prediction_id', 'cancer']].to_csv('/kaggle/working/submission.csv', index = False)

sub.head()

,prediction_id,cancer,site_id
0,10008_L,0,2
1,10008_R,0,2
